# POC : Comparaison entre TF-IDF + LogReg et ModernBERT (BERT L4 H256)

In [1]:
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import pandas as pd

/home/noureddine/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Chargement du dataset IMDb

In [2]:
dataset = load_dataset("imdb")
texts = dataset["train"]["text"] + dataset["test"]["text"]
labels = dataset["train"]["label"] + dataset["test"]["label"]

## Modèle baseline : TF-IDF + Logistic Regression

In [3]:
X_tfidf = TfidfVectorizer(max_features=10000).fit_transform(texts)
y = labels
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.5, random_state=42)
model_tfidf = LogisticRegression(max_iter=1000)
model_tfidf.fit(X_train, y_train)
preds_tfidf = model_tfidf.predict(X_test)
print(classification_report(y_test, preds_tfidf))

              precision    recall  f1-score   support

           0       0.90      0.88      0.89     12603
           1       0.88      0.91      0.89     12397

    accuracy                           0.89     25000
   macro avg       0.89      0.89      0.89     25000
weighted avg       0.89      0.89      0.89     25000



## Modèle ModernBERT (BERT L4 H256)

In [7]:
from transformers import (
    AutoTokenizer, 
    BertForSequenceClassification, 
    Trainer, 
    TrainingArguments, 
    DataCollatorWithPadding, 
    EarlyStoppingCallback
)
from datasets import load_dataset
import torch
import numpy as np
import evaluate
from sklearn.metrics import classification_report

# 1. Charger le dataset IMDb
dataset = load_dataset("imdb")

# 2. Tokenizer pour ModernBERT
tokenizer = AutoTokenizer.from_pretrained("google/bert_uncased_L-4_H-256_A-4")

def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=512
    )

# 3. Appliquer le tokenizer
tokenized = dataset.map(tokenize_function, batched=True)
tokenized = tokenized.remove_columns(["text"]).rename_column("label", "labels").with_format("torch")

# 4. Datasets
train_ds = tokenized["train"]
test_ds = tokenized["test"]

# 5. Charger le modèle ModernBERT
model = BertForSequenceClassification.from_pretrained("google/bert_uncased_L-4_H-256_A-4", num_labels=2)

# 6. Accuracy
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)

# 7. Arguments d’entraînement améliorés
training_args = TrainingArguments(
    output_dir="./bert_results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=4,
    weight_decay=0.01,
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    lr_scheduler_type="cosine",
    fp16=torch.cuda.is_available(),  
)

# 8. Trainer + early stopping
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

# 9. Entraînement
trainer.train()

# 10. Évaluation
preds = trainer.predict(test_ds)
print(classification_report(test_ds["labels"], np.argmax(preds.predictions, axis=1)))


Map: 100%|██████████| 50000/50000 [00:20<00:00, 2450.89 examples/s]
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-4_H-256_A-4 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/noureddine/.local/lib/python3.12/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_6133/4174112585.py:66: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.301000,0.286975,0.880760
2,0.245100,0.274347,0.889680
3,0.219000,0.253606,0.902880
4,0.185800,0.260644,0.903080


              precision    recall  f1-score   support

           0       0.90      0.90      0.90     12500
           1       0.90      0.90      0.90     12500

    accuracy                           0.90     25000
   macro avg       0.90      0.90      0.90     25000
weighted avg       0.90      0.90      0.90     25000

